# House Price Prediction with MLFLOW

In [ ]:
import mlflow
import pandas as pd
import mlflow.sklearn
from mlflow.models import infer_signature
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

In [ ]:
housing = fetch_california_housing()
print(housing)

In [ ]:
df = pd.DataFrame(housing.data,columns=housing.feature_names)
df["Price"] = housing.target

In [ ]:
df

In [ ]:
X = df.drop(columns=["Price"])
y = df["Price"]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [ ]:
from urllib.parse import urlparse


In [ ]:
signature = infer_signature(X_train,y_train)
signature

In [ ]:
## Hyperparameter Tuning
def hyperparameter_tuning(model,X_train,y_train,param_grid):
    model =  model
    grid_search = GridSearchCV(estimator=model,param_grid=param_grid,cv=3,verbose=2,scoring="neg_mean_squared_error")
    grid_search.fit(X_train,y_train)
    return grid_search

In [ ]:
rf_param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2],
    "max_features": ["auto", "sqrt", "log2"],
    "bootstrap": [True, False],
    "random_state": [42]
}

lr_param_grid = {
    "fit_intercept": [True, False],
    "copy_X": [True, False],
    "n_jobs": [None, -1],
    "positive": [False, True]
}
rf = RandomForestRegressor()
lr = LinearRegression()

In [ ]:
###  Loop through models and log each run
import mlflow.sklearn

# Tracking uri
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
# It extracts the URI scheme to determine if the MLflow tracking store is local (file) or remote (e.g., http), so the code knows whether model registration is supported.
tracking_uri_type_store=urlparse(mlflow.get_tracking_uri()).scheme
for model, param_grid in [(rf, rf_param_grid), (lr, lr_param_grid)]:
    with mlflow.start_run():
        grid_search = hyperparameter_tuning(model, X_train, y_train, param_grid)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)

        #  Log all best parameters
        mlflow.log_params(grid_search.best_params_)
        
        #  Add tags for clarity
        mlflow.set_tag("model", model.__class__.__name__)

        # Log metric
        mlflow.log_metric("mse", mse)
        
        #  Log model itself
        mlflow.sklearn.log_model(best_model, artifact_path="model")

        
        
        if tracking_uri_type_store != "file":
            mlflow.sklearn.log_model(best_model,"model",registered_model_name=f"Best_{model.__class__.__name__}_model")
        else:
            mlflow.sklearn.log_model(best_model,"model",signature=signature)
        print(f"Best Hyperparameters: {grid_search.best_params_}")
        print(f"Mean Squared Error: {mse}")

In [ ]:
print("RF Best params:", grid_search.best_params_)
print("RF MSE:", mse)
